In [1]:
from src import StudyGen

In [2]:
study = StudyGen(path_configuration='config.yaml', path_master='master.yaml')

In [3]:
study.generate_all()

['b', 'c'] odict_values([<class 'float'>, <class 'float'>])
ICI OrderedDict([('b', <class 'float'>), ('c', <class 'float'>)])
ICI2 OrderedDict([('a', <class 'float'>), ('b', <class 'float'>)])

['a', 'bc'] odict_values([<class 'float'>, <class 'float'>])
ICI OrderedDict([('a', <class 'float'>), ('bc', <class 'float'>)])
ICI2 OrderedDict([('a', <class 'float'>), ('b', <class 'float'>)])

['a_bc'] odict_values([<class 'float'>])
ICI OrderedDict([('a_bc', <class 'float'>)])
ICI2 OrderedDict([('a', <class 'float'>)])
import math
['fact_a_bc'] odict_values([typing.Any])
ICI OrderedDict([('fact_a_bc', typing.Any)])
ICI2 OrderedDict([('output', typing.Any)])
import numpy as np
from typing import Any
import math
import numpy as np
from typing import Any
['x', 'y'] odict_values([<class 'float'>, <class 'float'>])
ICI OrderedDict([('x', <class 'float'>), ('y', <class 'float'>)])
ICI2 OrderedDict([('b', <class 'float'>), ('c', <class 'float'>)])
import numpy as np
['x_y', 'z'] odict_values([<clas

ValueError: Number of parameters is different. Previous: 3. New: 2